In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets
import torch.optim as optim
from torch.autograd import Variable
from collections import OrderedDict
from torch.nn import init

from torch.utils.data import DataLoader, Dataset

cuda = torch.cuda.is_available()
cuda

True

In [2]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    uniqueID_list = []
    for root, directories, filenames in os.walk(datadir):
        b = -1
        for filename in filenames:
            if filename.endswith('.jpg'):
                if(filename[0]+filename[1]=='._'):
                    filename = filename[2:]
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])
                a = root.split('/')[-1]
                if a!=b:
                    uniqueID_list.append(a)
                b=a
                
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n)))
    reverse_dict = dict(zip(range(class_n),uniqueID_list))
    label_list = [target_dict[ID_key] for ID_key in ID_list]
    
    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n, reverse_dict

In [29]:
class Simple(nn.Module):
    def __init__(self, n_in = 512*3, n_out = 2300):
        super(Simple, self).__init__()
        #self.conv1 = nn.Conv1d(in_channels=1, out_channels=256, kernel_size=8, stride=1)
        
        self.fc1 = nn.Linear(n_in, n_in*2, bias = True)
        #self.fc1 = nn.Linear(256, n_in, bias = True)
        self.dropout1 = nn.Dropout(p=0.1)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(n_in*2)
        #self.bn1 = nn.BatchNorm1d(n_in)
        
        self.fc2 = nn.Linear(n_in*2, n_in*4, bias = True)
        #self.fc2 = nn.Linear(n_in, n_out, bias = True)
        self.dropout2 = nn.Dropout(p=0.2)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm1d(n_in*4)

        self.fc0 = nn.Linear(n_in*4, n_in*4, bias = True)
        self.relu0 = nn.ReLU()
        self.bn0 = nn.BatchNorm1d(n_in*4)
        
        self.fc3 = nn.Linear(n_in*4, n_out, bias = True)
        #self.logprob = nn.LogSoftmax(dim=1)
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
  
    def forward(self, x):
        #x = x.unsqueeze(1)
        #x = self.conv1(x)  #(N, C, K)
        #x = x.mean([2])
        x = self.fc1(x)
        #x = self.dropout1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        
        x = self.fc2(x)
        #x = self.dropout2(x)
        x = self.relu2(x)
        x = self.bn2(x)
        
        x = self.fc0(x)
        x = self.relu0(x)
        x = self.bn0(x)
        
        x = self.fc3(x)
        #x = self.logprob(x)
        
        return x

In [30]:
network = Simple()
network.load_state_dict(torch.load('embedding_MLP4.pth'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network.to(device)
network.eval()

Simple(
  (fc1): Linear(in_features=1536, out_features=3072, bias=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (relu1): ReLU()
  (bn1): BatchNorm1d(3072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=3072, out_features=6144, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (relu2): ReLU()
  (bn2): BatchNorm1d(6144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc0): Linear(in_features=6144, out_features=6144, bias=True)
  (relu0): ReLU()
  (bn0): BatchNorm1d(6144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=6144, out_features=2300, bias=True)
)

In [31]:
class PDataset(Dataset):
    def __init__(self, data_file_list):
        self.data_file_list = data_file_list
        
        self.data1 = np.load(data_file_list[0])
        self.data2 = np.load(data_file_list[1])
        self.data3 = np.load(data_file_list[2])

    
    def __len__(self):
        return self.data1.shape[0]
    
    def __getitem__(self, index):
        stack1 = np.hstack((self.data1[index], self.data2[index]))
        embedding = np.hstack((stack1, self.data3[index]))
        embedding = torch.from_numpy(embedding)

        return embedding

In [32]:
testset = PDataset(["shufflenet_test_embedding.npy", "MobileNet_test_embedding.npy", "resnet_test_embedding.npy"])
test_dataloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=3,drop_last=False)

In [33]:
_,_, _, reverse_dict = parse_data('train_data/medium')

#Images		#Labels
822154		2300


In [34]:
pred_list = []
for batch_num, (feats) in enumerate(test_dataloader):
    feats = feats.to(device)
    prediction = network(feats.float())
    _, pred_labels = torch.max(F.softmax(prediction, dim=1), 1)
    #pred_labels = pred_labels.view(-1)
    pred_list.append(pred_labels.cpu().numpy())
    del feats
    
pred_list = np.array(pred_list)
real_list = np.zeros_like(pred_list)
for i in range(pred_list.shape[0]):
    real_list[i] = reverse_dict[int(pred_list[i])]
    


In [35]:
print(real_list[0:100])

[[ 947]
 [ 317]
 [1277]
 [2115]
 [ 863]
 [ 384]
 [1484]
 [ 172]
 [ 702]
 [ 720]
 [2057]
 [2152]
 [1722]
 [1690]
 [1528]
 [2009]
 [ 489]
 [1879]
 [1127]
 [1786]
 [1924]
 [1452]
 [1638]
 [  42]
 [2067]
 [2100]
 [ 282]
 [1601]
 [1619]
 [ 611]
 [1428]
 [1457]
 [1385]
 [1341]
 [ 911]
 [ 574]
 [2158]
 [2177]
 [1050]
 [ 297]
 [ 597]
 [1810]
 [ 379]
 [1371]
 [ 716]
 [1216]
 [ 743]
 [1997]
 [1534]
 [1049]
 [ 191]
 [2262]
 [1868]
 [1472]
 [1886]
 [2230]
 [   0]
 [2203]
 [1821]
 [ 883]
 [ 219]
 [2086]
 [1552]
 [ 308]
 [1815]
 [1655]
 [1782]
 [1620]
 [ 402]
 [ 602]
 [2034]
 [1263]
 [ 967]
 [1895]
 [1002]
 [2111]
 [1033]
 [ 952]
 [1917]
 [1132]
 [2194]
 [  42]
 [ 385]
 [1564]
 [ 619]
 [2178]
 [ 261]
 [ 846]
 [1334]
 [ 731]
 [1415]
 [1307]
 [1965]
 [1311]
 [ 495]
 [1083]
 [1629]
 [ 971]
 [1586]
 [ 536]]


In [36]:
np.savetxt("kaggle_result4.csv",real_list,delimiter=",")

In [11]:
print(reverse_dict)

{0: '857', 1: '2071', 2: '575', 3: '819', 4: '2254', 5: '629', 6: '2136', 7: '1001', 8: '2230', 9: '480', 10: '1084', 11: '1303', 12: '1588', 13: '342', 14: '1884', 15: '1882', 16: '1408', 17: '745', 18: '1157', 19: '464', 20: '669', 21: '1393', 22: '1749', 23: '1271', 24: '2161', 25: '1576', 26: '1283', 27: '1493', 28: '708', 29: '1189', 30: '1196', 31: '2234', 32: '1535', 33: '499', 34: '642', 35: '2214', 36: '865', 37: '1459', 38: '422', 39: '696', 40: '245', 41: '474', 42: '1251', 43: '1496', 44: '1295', 45: '74', 46: '578', 47: '1626', 48: '2133', 49: '98', 50: '1128', 51: '500', 52: '513', 53: '110', 54: '1429', 55: '2073', 56: '1959', 57: '25', 58: '963', 59: '1262', 60: '954', 61: '813', 62: '1212', 63: '1207', 64: '2209', 65: '1777', 66: '1979', 67: '1044', 68: '782', 69: '469', 70: '128', 71: '1668', 72: '320', 73: '1936', 74: '1642', 75: '1059', 76: '12', 77: '170', 78: '326', 79: '767', 80: '2227', 81: '2027', 82: '951', 83: '1123', 84: '1093', 85: '1422', 86: '2141', 87: '